In [ ]:
import logging

In [1]:
class MetricsClient:
    """
    타사 지표 전송 클라이언트
    """
    def send(self, metric_name, metric_value):
        if not isinstance(metric_name,str):
            raise TypeError("metric_name으로 문자열 타입을 사용해야 함")
        if not isinstance(metric_value,str):
            raise TypeError("metric_value로 문자열 타입을 사용해야 함")

        logging.info("%s 전송 값 = %s", metric_name, metric_value)

In [2]:
class Process:
    def __init__(self):
        self.client = MetricsClient() # 타사 지표 전송 클라이언트

    def process_interations(self, n_iterations):
        for i in range(n_iterations):
            result = self.run_process()
            self.client.send("iteration.".format(i), result)

- 메서드가 작다는 것은 테스트 가능성과 관련해 좋은 디자인이라고 할 수 있다.
- 마지막으로 필요한 부분만 테스트하기 위해 main 메서드에서 client를 직접 다루지 않고 래퍼(wrapper)메서드에 위임한다.

In [3]:
class WrappedClient:
    def __init__(self):
        self.client = MetricsClient()
    
    def send(self, metric_name, metric_value):
        return self.client.send(str(metric_name), str(metric_value))

In [4]:
class Process:
    def __init__(self):
        self.client = WrappedClient()

    def process_interations(self, n_iterations):
        for i in range(n_iterations):
            result = self.run_process()
            self.client.send("iteration.".format(i), result)
        

In [6]:
import unittest
from unittest.mock import Mock

In [ ]:
class TestWrappedClient(unittest.TestCase):
    def test_send_converts_types(self):
        wrapped_client = WrappedClient()
        wrapped_client.client = Mock()
        wrapped_client.send("value", 1)
        wrapped_client.client.send.assert_called_with("value", "1")